<center><h1> Lab04 sheet </h1></center>

**Student ID: 22670287** <br>
**Student Name: Xiaoyan Huang**<br>
**Date created: 21/04/2020**<br>
**Last modified: 28/04/2020**<br>

The main goal of Project1 is to compare the performances in AdaBoost Regressor and a Gradient Boosting Regressor.

# 1. Project1

## 1.1 Preprocess data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
whtwine = pd.read_csv("winequality-white.csv", sep = ";")

In [ ]:
whtwine.shape

In [ ]:
X = whtwine.iloc[:,:11]
y = whtwine['quality']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.15, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.describe()

In [ ]:
import matplotlib.pyplot as plt
X_mean = X_train.mean(axis=0)
xlab = list(X.columns)
ss = np.arange(len(xlab))
plt.bar(ss, X_mean)
plt.xticks(ss, xlab, rotation = 45)
plt.ylabel('Mean')
plt.title('the mean of each feature')
plt.show()

In [ ]:
from sklearn import preprocessing

In [ ]:
X_train_sc = preprocessing.scale(X_train)
X_test_sc = preprocessing.scale(X_test)

In [ ]:
X_train_sc.mean(axis=0), X_train_sc.std(axis=0)

- The whitewine data was split into training set and test set by a 85/15 proportion. There are 4163 instances in training set and the rest of instances in test set.

- After the inspection of data via `describe` function or the bar chart, the ranges of features are different. It is necessary to scale the data both in training and test set. The new variables *X_train_sc* and *X_test_sc* are scaled to have a mean of zero and a standard deviations of one.

## 1.2 AdaBoost Regressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
svr_clf = SVR(kernel="rbf")
ada_clf = AdaBoostRegressor(
    base_estimator=svr_clf,
    n_estimators=6, learning_rate=1, random_state=42)

In [ ]:
ada_clf.fit(X_train_sc, y_train)

In [ ]:
prd_value_test = ada_clf.predict(X_test_sc)
prd_value_train = ada_clf.predict(X_train_sc)

In [ ]:
int_prd_value_test = np.around(prd_value_test)
int_prd_value_train = np.around(prd_value_train)

- There is AdaBoost Regressor using SVM regressor with RBF kernel and it has only 6 estimators. However, the predictive values aren't integers due to the regressor model. Thus, using `np.around` converts output into integer for both training and test set.

### 1.2.1 Inspect Mean Square Error

In [ ]:
from sklearn.metrics import mean_squared_error
mse_test = mean_squared_error(y_test,int_prd_value_test)
print("Mean square error of final model in test data is %s " % mse_test)

In [ ]:
prd_value_irt = ada_clf.staged_predict(X_test_sc)
mes_irt = []
for i in prd_value_irt:
    int_prd_value_irt = np.around(i)
    mes_irt.append(mean_squared_error(y_test, int_prd_value_irt))
print("The MSEs of all the models in test set \n")
print(mes_irt)

In [ ]:
mse_train = mean_squared_error(y_train, int_prd_value_train)
print("Mean square error of final model in training data is %s " % mse_train)

In [ ]:
prd_value_irt2 = ada_clf.staged_predict(X_train_sc)
mes_irt2 = []
for i in prd_value_irt2:
    int_prd_value_irt2 = np.around(i)
    mes_irt2.append(mean_squared_error(y_train, int_prd_value_irt2))
print("The MSEs of all the models in training set \n")
print(mes_irt2)

In [ ]:
import matplotlib.pyplot as plt
x = [1,2,3,4,5,6]
plt.plot(x, mes_irt, marker="o", c='r', label="MSE of test")
plt.plot(x, mes_irt2, marker="*", c='b', label="MSE of train")
plt.legend()
plt.xlabel("iteration time")
plt.ylabel("MSE")
plt.show()

- Compared the mean square error of final models on traning set with test set, the fact is that the Mean Square Error of training data is lower than test data (0.45 versus 0.58). 
- And then, the MSEs of all the intermediate models are calculated via `staged_predict` function on both datasets, and these results are exhibited by a line chart. From the above plot, we know that the MSEs are both descreasing as iteration time increases and these decrease significantly when iteration time is equal to 3.

### 1.2.2  Inspect Raw Errors

In [ ]:
errors_ada_test = pd.DataFrame()
errors_ada_test['predictive'] = int_prd_value_test
errors_ada_test['y_test'] = y_test.tolist()
errors_ada_test['errors'] = errors_ada_test['predictive'] - errors_ada_test['y_test']

In [ ]:
cnt_test = errors_ada_test['errors'].value_counts()
cnt_test = cnt_test.sort_index(ascending=True)
cnt_test

In [ ]:
xlab = cnt_test.index
ss = np.arange(len(xlab))
plt.bar(ss, cnt_test)
plt.xticks(ss, xlab)
plt.ylabel('Count')
plt.title('the raw errors of the final model for Adaboost on test set')
plt.show()

In [ ]:
errors_ada_train = pd.DataFrame()
errors_ada_train['predictive'] = int_prd_value_train
errors_ada_train['y_train'] = y_train.tolist()
errors_ada_train['errors'] = errors_ada_train['predictive'] - errors_ada_train['y_train']

In [ ]:
cnt_train = errors_ada_train['errors'].value_counts()
cnt_train = cnt_train.sort_index(ascending=True)
cnt_train

In [ ]:
xlab = cnt_test.index
ss = np.arange(len(xlab))
plt.bar(ss, cnt_train)
plt.xticks(ss, xlab)
plt.ylabel('Count')
plt.title('the raw errors of the final model for Adaboost on train set')
plt.show()

- In order to inspect raw errors, the *errors_ada_test* and *errors_ada_train* dataframe are created to store the predictive value, ground true and raw errors. And then, these are counted and exhibited by bar charts.
- In terms of test set, the regressor correctly predicts the quality rating 414 times. Meanwhile, the regressor overestimates (or underestimates) the quality rating by one (or minus one) 138 times (or 150 times). Even worse, it overestimates the quality rating by 3 one times.
- For the training set, the regressor performs well. It correctly predicts the quality rating with 2583 times, and overestimates (or underestimates) the quality rating by one (or minus one) 686 times (or 791 times).
- Thus, there are some problems with adaboost regressor that it is easy to overestimate or underestimate values even though estimating the quality rating by one or minus one seems not bad.

## 1.3 GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=6, learning_rate=1, random_state=42)
gbr.fit(X_train_sc, y_train)

In [ ]:
gbr_prd_test = gbr.predict(X_test_sc)
gbr_prd_train = gbr.predict(X_train_sc)

In [ ]:
int_gbr_prd_test = np.around(gbr_prd_test)
int_gbr_prd_train = np.around(gbr_prd_train)

- It is a GradientBoostingRegressor with only 6 estimators. There are same processes transforming predictive values into integers.


### 1.3.1 Inspect Mean Square Error

In [ ]:
gbr_mse_test = mean_squared_error(y_test, int_gbr_prd_test)
print("Mean square error of final model in test data is %s " % gbr_mse_test)

In [ ]:
gbr_prd_value_irt = gbr.staged_predict(X_test_sc)
gbr_mes_irt = []
for i in gbr_prd_value_irt:
    int_gbr_prd_value_irt = np.around(i)
    gbr_mes_irt.append(mean_squared_error(y_test, int_gbr_prd_value_irt))
print("The MSEs of all the models in test set \n")
print(gbr_mes_irt)

In [ ]:
gbr_mse_train = mean_squared_error(y_train, int_gbr_prd_train)
print("Mean square error of final model in training data is %s " % gbr_mse_train)

In [ ]:
gbr_prd_value_irt2 = gbr.staged_predict(X_train_sc)
gbr_mes_irt2 = []
for i in gbr_prd_value_irt2:
    int_gbr_prd_value_irt2 = np.around(i)
    gbr_mes_irt2.append(mean_squared_error(y_train, int_gbr_prd_value_irt2))
print("The MSEs of all the models in train set \n")
print(gbr_mes_irt2)

In [ ]:
import matplotlib.pyplot as plt
x = [1,2,3,4,5,6]
plt.plot(x, gbr_mes_irt, marker="o", c="r", label="MSE of test")
plt.plot(x, gbr_mes_irt2, marker="*", c="b", label="MSE of train")
plt.legend()
plt.xlabel("iteration time")
plt.ylabel("MSE")
plt.show()

- Compared the mean square error of final models on traning set with test set, they are similar 0.68 and 0.60 respectively.
- The MSEs of all the intermediate models are computed on both datasets and are exhibited by a line chart. we know the two lines decrease significantly as iteration time increases.
- when iteration time is equal to 5, it seems better for test set.

### 1.3.2  Inspect Raw Errors

In [ ]:
errors_gbr_test = pd.DataFrame()
errors_gbr_test['predictive'] = int_gbr_prd_test
errors_gbr_test['y_test'] = y_test.tolist()
errors_gbr_test['errors'] = errors_gbr_test['predictive'] - errors_gbr_test['y_test']

In [ ]:
cnt_test2 = errors_gbr_test['errors'].value_counts()
cnt_test2 = cnt_test2.sort_index(ascending=True)
cnt_test2

In [ ]:
xlab = cnt_test2.index
ss = np.arange(len(xlab))
plt.bar(ss, cnt_test2)
plt.xticks(ss, xlab)
plt.ylabel('Count')
plt.title('the raw errors of the final model for Gradient Boosting on test set')
plt.show()

In [ ]:
errors_gbr_train = pd.DataFrame()
errors_gbr_train['predictive'] = int_gbr_prd_train
errors_gbr_train['y_test'] = y_train.tolist()
errors_gbr_train['errors'] = errors_gbr_train['predictive'] - errors_gbr_train['y_test']

In [ ]:
cnt_train2 = errors_gbr_train['errors'].value_counts()
cnt_train2 = cnt_train2.sort_index(ascending=True)
cnt_train2

In [ ]:
xlab = cnt_train2.index
ss = np.arange(len(xlab))
plt.bar(ss, cnt_train2)
plt.xticks(ss, xlab)
plt.ylabel('Count')
plt.title('the raw errors of the final model for Gradient Boosting on train set')
plt.show()

- In terms of test set, the regressor correctly predicts the quality rating 360 times. Meanwhile, the regressor overestimates (or underestimates) the quality rating by one (or minus one) 160 times (or 180 times). However, it overestimates the quality rating by 3 four times.
- For the training set, the regressor performs not good. It correctly predicts the quality rating with only 2222 times, and overestimates (or underestimates) the quality rating by one (or minus one) 842 times (or 942 times). There are 12 samples which are wrongly predicted the quality rating by 3,(-3) or 4.
- Thus, the Gradient Boosting regressor performs not well in this case both on training and test set.

## 1.4 Comparison

### 1.4.1 Compare MSEs

In [ ]:
x = [1,2,3,4,5,6]
plt.plot(x, mes_irt, marker="o", c='r', label="Ada test")
plt.plot(x, mes_irt2, marker="o", c='b', label="Ada train")
plt.plot(x, gbr_mes_irt, marker="*", c="y", label="gbr test")
plt.plot(x, gbr_mes_irt2, marker="*", c="g", label="gbr train")
plt.legend(prop={'size': 7.5})
plt.xlabel("iteration time")
plt.ylabel("MSE")
plt.show()

- The MSEs are put in a plot. The AdaBoost regressor fits the data well that it is lower in MSE values for both training and test set. In contrast, the gradientBoost regressor is underfitting, the MSE is slightly higher than previous model. But the gap between training set and test set is smaller.

### 1.4.2 Compare Raw Errors


- If we assume that the quality rating by one or minus one is not bad. The adaboost regressor did a good job on trainin set, but there are **23** samples overestimated or underestimated over 1 on test set.
- Under the same assumptions, for GradientBoosting regressor, it performed not well on the test set because of underfitting. In the training set, there are only **2222** predicted correctly (**2583** in adaboost regressor), the performance in training set is not enough. Besides, only **360** samples predicted correctly (**414** in adaboost model). It also predicted badly **35** samples these error over by 1 in the test set.
- In conclusion, Adaboost performed well but GradientBoosting is underfitting.